**Datset Loading**

In [ ]:
#Connecting the cloud to download the images
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#Importing the required libraries
import os
import numpy as np
import cv2
from glob import glob
from google.colab.patches import cv2_imshow
from scipy.ndimage import gaussian_filter
from scipy.ndimage.filters import median_filter
from skimage.morphology import disk
from skimage import color
import pickle as pkl
from sklearn.metrics import jaccard_score
from sklearn.cluster import KMeans
from scipy import ndimage
from skimage.morphology import remove_small_objects
import matplotlib.pyplot as plt

In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):

    train_x = sorted(glob(os.path.join(path, "/content/drive/MyDrive/Colab Notebooks/ISIC/images", "*.jpg")))
    masks = sorted(glob(os.path.join(path, "/content/drive/MyDrive/Colab Notebooks/ISIC/masks", "*.png")))

    return train_x, masks

In [ ]:
#Loading the original training images and masks
if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)

    """ Load the data """
    data_path = "/content/drive/MyDrive/Colab Notebooks/ISIC"
    (train_x, masks) = load_data(data_path)

    print(f"Train: {len(train_x)}")
    

Train: 200


**Preprocessing with Hair Removal**

In [ ]:
######## Create Rotating and Tilted Structuring Element ##########
def createTiltedStructuringElements(width, height, n):
  # width = 11
  # height = 1
  # n = 30
  base = np.zeros((width, width), np.uint8)
  start = int(width/2-height/2)
  end = int(width/2+height/2)
  for k in range(start+1, end+1):
    cv2.line(base,(0,k),(width,k),(255,255,255))
  SE_list = []
  SE_list.append(base)
  angle_step = 360.0/n
  for i in range(n):
    rows = base.shape[0]
    cols = base.shape[1]
    M = cv2.getRotationMatrix2D((cols/2.0, rows/2.0), i*angle_step, 1.0)
    SE = cv2.warpAffine(base, M, (width, width),  cv2.INTER_NEAREST);
    SE_list.append(SE)
  return SE_list

In [ ]:
#Creating a function for removing the hairs
def image_preprocessing(src_image, SE_list):
  
  #Removing noise from the image
  median_image = median_filter(src_image, 3)
  #Converting to grayscale for appling gs morphology
  gray_image = cv2.cvtColor(median_image, cv2.COLOR_BGR2GRAY)
  #Inverting the intentisities of the pixels before using tophat transform
  inv_image = cv2.bitwise_not(gray_image)
  #Defining a matrix to store the sum of tophats
  sumOfTopHats = np.zeros((inv_image.shape[0], inv_image.shape[1]), np.uint16)

  #Applying a sum of tophat transform with different rotated SE to the images
  for SE in SE_list:
    tophat_image = cv2.morphologyEx(inv_image, cv2.MORPH_TOPHAT, SE)
    tophat_image = np.uint16(tophat_image)
    sumOfTopHats += tophat_image
  #Normalizing the intensities values of the pixels
  cv2.normalize(sumOfTopHats, sumOfTopHats, 0, 255, cv2.NORM_MINMAX);
  #Converting back to uint8 for visualization
  sumOfTopHats = np.uint8(sumOfTopHats)
  #Changing againg the pixels intensities before thresholding
  inv_tophat = cv2.bitwise_not(sumOfTopHats)
  #Thresholding the image to create a mask
  ret, thres_image = cv2.threshold(sumOfTopHats, 10, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

  #Applying morphological operations for removing remaining dark objects
  disk_kernel = disk(5)
  closed_image1 = cv2.morphologyEx(thres_image, cv2.MORPH_CLOSE, disk_kernel)
  # dil_kernel = disk(5)
  dil_kernel = np.ones((3,3),np.uint8)
  dilation_image = cv2.dilate(closed_image1,dil_kernel,iterations = 1)

  #Inpainting with the dilated mask for removing remaining dark objects
  inpainted_image = cv2.inpaint(src_image,dilation_image,20,cv2.INPAINT_TELEA)

  return inpainted_image

In [ ]:
#Defining a circular mask for the FOV removal algorithm
def create_circular_mask(h, w, center, radius):
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    return mask

In [ ]:
#Defining a function for FOV removal
def remove_black_border(inp_image):
  with_border_img = inp_image
  #Defining the size of the circular mask
  h, w = with_border_img.shape[:2]
  center = (int(w/2), int(h/2))
  radius = min(center[0], center[1], w-center[0], h-center[1])
  mask = create_circular_mask(h,w,center,radius)
  masked_img =  with_border_img.copy()
  #Counting the pixels intensities
  count_black = np.count_nonzero(masked_img[~mask] < 50)
  count_white = np.count_nonzero(masked_img[~mask] >= 50)
  count_total = count_black + count_white
  check = count_black/count_total*100
  diff = 100
  #Setting a condition for identifying whether there is a FOV in the image
  if(check<10):
    return inp_image
  else:
  #Counting the intensities values of the pixels and subtracting the circular mask from the original image
  #until there is no more black ring. 
    while diff>10:
      radius = radius-20 #The radius is reduced on each iteration to subtract remaining black pixels
      mask = create_circular_mask(h,w,center,radius)
      #Initial pixels intensities values on each iteration
      old_black = count_black
      old_white = count_white
      old_total = count_total
      #Counting the intensities values on each iteration
      count_black = np.count_nonzero(masked_img[~mask] < 50)
      count_white = np.count_nonzero(masked_img[~mask] >= 50)
      count_total = count_black + count_white
      #Checking if the stopping condition is met
      diff_black = abs(count_black-old_black)
      diff_white = abs(count_white-old_white)
      diff_total = abs(count_total-old_total)
      diff = diff_black/(diff_total+1)*100
  masked_img[~mask] = 185
  return  masked_img


**K-Means Clustering**

In [ ]:
def kmeans(prep_img):
  #Converting to RGB
  newRGB=cv2.cvtColor(prep_img,cv2.COLOR_BGR2RGB)
  
  
  #Flattening our image from 3D to 2D ((w*h),RGB)
  flat_img=newRGB.reshape((-1,3))

  #Converting to float values
  flat_img=np.float32(flat_img)
  #print(flat_img.shape)

  #Define stopping criteria (either i>100 or k<e=0.2)
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)

  # number of clusters (k)
  k = 2

  #labels = cluster label for each pixel (up to k)/ center points
  ret, labels, centers = cv2.kmeans(flat_img, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

  #Back to uint8
  centers = np.uint8(centers)

  #median_filter
  centers = median_filter(centers, 3)  

  #flatten labels
  labels = labels.flatten()

  #All pixels to color of the centers
  segmented_img = centers[labels.flatten()]

  #Back to the original img dim
  segmented_img=segmented_img.reshape(prep_img.shape)
  
  #Converting to Grayscale before thresholding
  GS_seg=cv2.cvtColor(segmented_img,cv2.COLOR_RGB2GRAY)

  #Creating a binary mask
  ret,otsu_img=cv2.threshold(GS_seg,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)


  return otsu_img

In [ ]:
def image_segmentation(inpainted_image):
  
  # ms_image = cv2.pyrMeanShiftFiltering(inpainted_image, 10, 30, 0)
  ms_image = inpainted_image

  km_image = kmeans(ms_image)
  # if flag_border==1:
  #   img = km_image
  #   kernel = np.ones((3,3),np.uint8)
  #   marker = img.copy()
  #   marker[1:-1,1:-1]=0
  #   while True:
  #       tmp=marker.copy()
  #       marker=cv2.dilate(marker, kernel)
  #       marker=cv2.min(img, marker)
  #       difference = cv2.subtract(marker, tmp)
  #       if cv2.countNonZero(difference) == 0:
  #           break

  #   mask=cv2.bitwise_not(marker)
  #   out=cv2.bitwise_and(img, mask)
  #   return out
  # else:
  return km_image
   


*Post-Processing*

In [ ]:
#Defining a function for extranting the largest connected component
def extract_largest_component(image):
  #Searching for contours (connected componets, shapes) on the images
  contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  if len(contours)==0: #If there is no contours, it means that the image is completely black
    contour_image= image 
  else:
    #Calculating the area of all the contours 
    areas= [cv2.contourArea(i) for i in contours]
    #Extracting the shape with the largest area
    largest_contour_idx= areas.index(max(areas))
    #Creating a temporaty black image
    black_image= np.zeros_like(image,np.uint8)
    #Printing the extracted shape (largest connected component) on the black image.
    contour_image= cv2.drawContours(black_image,contours,largest_contour_idx,(255,255,255),-1)
  return contour_image

**WorkFlow**

In [ ]:
#Rezising the training images and masks
X_train = []
X_masks = []
for data in train_x:
  image = cv2.imread(data)
  # print('Original Dimensions : ',img.shape)
  dim = (256, 256)
  # resize image
  resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
  X_train.append(resized)

for data in masks:
  image = cv2.imread(data)
  # print('Original Dimensions : ',img.shape)
  dim = (256, 256)
  # resize image
  resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
  X_masks.append(resized)

In [ ]:
## Creating Tilted Structuring Element for sum of TopHats
SE_list = []
SE_list = createTiltedStructuringElements(11, 1, 40)

In [ ]:
#Pre-processing: hair removal
preprocessed_image = []
for data in X_train:
  image = image_preprocessing(data, SE_list)
  preprocessed_image.append(image)

In [ ]:
# preprocessed_image = np.load('/content/drive/MyDrive/Colab Notebooks/ISIC 2017/Results/preprocessed_images_200.npy')

In [ ]:
#Removing the FOV, segmenting the image and extrating the largest connected component.
segmented_image = []
for data in preprocessed_image:
  clean_img = remove_black_border(data)
  seg_img = image_segmentation(clean_img)
  conn_img = extract_largest_component(seg_img)
  segmented_image.append(conn_img)

Change the FILE Name

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/ISIC/preprocessed_images_train_256.npy', 'wb') as f:
    np.save(f, preprocessed_image)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/ISIC/segmented_images_train_256.npy', 'wb') as f:
    np.save(f, segmented_image)

**Jaccard and Dice Score**

In [ ]:
#Defining a JC function to evaluate the masks generated.
def jaccard(im1, im2, empty_score=1.0):
    im1 = np.asarray(im1).astype(bool)
    im2 = np.asarray(im2).astype(bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    im_sum = im1.sum() + im2.sum()
    if im_sum == 0:
        return empty_score

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)
    union = np.logical_or(im1, im2)

    return intersection.sum() / float(union.sum())

In [ ]:
#Defining a DSC function to evaluate the masks generated.
def dice(im1, im2, empty_score=1.0):
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    im_sum = im1.sum() + im2.sum()
    if im_sum == 0:
        return empty_score

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / im_sum

In [ ]:
#Evaluating the handcrafted binary masks.
#segmented_image= np.load('/content/drive/MyDrive/Colab Notebooks/ISIC/preprocessed_images.npy')
jaccard_list = []
dice_list = []
for i in range(len(segmented_image)):
  gray_masks = cv2.cvtColor(X_masks[i], cv2.COLOR_BGR2GRAY)
  image = segmented_image[i]
  # print(i)
  # cv2_imshow(X_train[i])
  # cv2_imshow(image)
  # cv2_imshow(gray_masks)
  # score = jaccard_score(gray_masks.flatten(), image.flatten(), average='micro')
  score = jaccard(image,gray_masks)
  dice_score = dice(image, gray_masks)
  # print(score)
  jaccard_list.append(score)
  dice_list.append(dice_score)
avg = np.mean(jaccard_list)
avg_dice = np.mean(dice_list)
print(avg)
print(avg_dice)

0.7040237282689381
0.8108662980882361


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until
